## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [26]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.0970,-79.2353,49.59,63,20,8.05,few clouds
1,1,Xilitla,MX,21.3333,-98.9667,74.71,82,49,2.80,scattered clouds
2,2,Atuona,PF,-9.8000,-139.0333,79.07,78,21,16.08,light rain
3,3,Mar Del Plata,AR,-38.0023,-57.5575,57.09,87,0,8.01,clear sky
4,4,Punta Arenas,CL,-53.1500,-70.9167,46.51,87,0,11.50,clear sky


In [27]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature (in Fahrenheit) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature (in Fahrenheit) you would like for your trip? "))

What is the minimum temperature (in Fahrenheit) you would like for your trip? 70
What is the maximum temperature (in Fahrenheit) you would like for your trip? 90


In [28]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Xilitla,MX,21.3333,-98.9667,74.71,82,49,2.80,scattered clouds
2,2,Atuona,PF,-9.8000,-139.0333,79.07,78,21,16.08,light rain
5,5,Kalianget,ID,-7.0500,113.9333,85.62,69,91,3.49,overcast clouds
8,8,Thinadhoo,MV,0.5333,72.9333,82.65,72,31,3.40,scattered clouds
10,10,Taunggyi,MM,20.7833,97.0333,76.26,48,97,2.48,overcast clouds
11,11,Vaini,TO,-21.2000,-175.2000,84.36,70,20,10.36,few clouds
13,13,Merauke,ID,-8.4667,140.3333,87.67,62,73,5.23,broken clouds
15,15,Kapaa,US,22.0752,-159.3190,82.38,77,20,16.11,few clouds
16,16,Hithadhoo,MV,-0.6000,73.0833,82.53,69,38,2.75,scattered clouds
17,17,Hilo,US,19.7297,-155.0900,77.29,88,100,6.91,haze


In [30]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [31]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.replace("", np.NaN).dropna()
clean_df.count()

City_ID                266
City                   266
Country                266
Lat                    266
Lng                    266
Max Temp               266
Humidity               266
Cloudiness             266
Wind Speed             266
Current Description    266
dtype: int64

In [32]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Xilitla,MX,74.71,scattered clouds,21.3333,-98.9667,
2,Atuona,PF,79.07,light rain,-9.8000,-139.0333,
5,Kalianget,ID,85.62,overcast clouds,-7.0500,113.9333,
8,Thinadhoo,MV,82.65,scattered clouds,0.5333,72.9333,
10,Taunggyi,MM,76.26,overcast clouds,20.7833,97.0333,
11,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,
13,Merauke,ID,87.67,broken clouds,-8.4667,140.3333,
15,Kapaa,US,82.38,few clouds,22.0752,-159.3190,
16,Hithadhoo,MV,82.53,scattered clouds,-0.6000,73.0833,
17,Hilo,US,77.29,haze,19.7297,-155.0900,


In [33]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
print("----------------------------")
print("End of search")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
----------------------------
End of search


In [37]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Xilitla,MX,74.71,scattered clouds,21.3333,-98.9667,Rio Tancuilin
2,Atuona,PF,79.07,light rain,-9.8000,-139.0333,Villa Enata
5,Kalianget,ID,85.62,overcast clouds,-7.0500,113.9333,Pesanggrahan Garam
8,Thinadhoo,MV,82.65,scattered clouds,0.5333,72.9333,
10,Taunggyi,MM,76.26,overcast clouds,20.7833,97.0333,True Treasure Hotel
...,...,...,...,...,...,...,...
693,Progreso,MX,83.10,scattered clouds,21.2833,-89.6667,Playa Linda Hotel
698,Maraa,BR,72.41,overcast clouds,-1.8333,-65.3667,
699,Issia,CI,72.82,light rain,6.4922,-6.5856,Résidence Hôtel Ange Anaé
700,Barinas,VE,80.64,overcast clouds,8.6226,-70.2075,Hotel Mastranto Suite


In [38]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace("", np.NaN).dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Xilitla,MX,74.71,scattered clouds,21.3333,-98.9667,Rio Tancuilin
2,Atuona,PF,79.07,light rain,-9.8000,-139.0333,Villa Enata
5,Kalianget,ID,85.62,overcast clouds,-7.0500,113.9333,Pesanggrahan Garam
10,Taunggyi,MM,76.26,overcast clouds,20.7833,97.0333,True Treasure Hotel
11,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
690,Doka,SD,84.15,overcast clouds,13.5167,35.7667,Um Rakuba Refugee Camp
693,Progreso,MX,83.10,scattered clouds,21.2833,-89.6667,Playa Linda Hotel
699,Issia,CI,72.82,light rain,6.4922,-6.5856,Résidence Hôtel Ange Anaé
700,Barinas,VE,80.64,overcast clouds,8.6226,-70.2075,Hotel Mastranto Suite


In [39]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))